In [38]:
import re
import numpy as np

In [50]:
disconnected_line = np.array([])
disconnected_gen = np.array([])
load_shedding_matrix = np.zeros(2)
p1 = re.compile(r"Line(.*)\sOvercurrent Relay.ElmRelay",re.S)
p2 = re.compile(r"SYM(.*)\sOFGT Relay.ElmRelay",re.S)
p3 = re.compile(r"SYM(.*)\sOVGT Relay.ElmRelay",re.S)
p4 = re.compile(r"Load(.*)\sUFLS Relay.ElmRelay",re.S)
p5 = re.compile(r"SYM(.*).ElmSym",re.S)
timep = re.compile(r"t=(.*)\ss",re.S)

In [51]:
f = open('report.txt','r')
load_power = [450+217.95j,80+38.75j]
shedding_percent = 0.25
for line in f.readlines():
    temp = line.strip()
    print('Read line: ',temp)
    if temp.find('Overcurrent Relay.ElmRelay') >= 0:
        target_line = int(re.findall(p1,temp)[0])
        print('Target line:',target_line)
        disconnected_line = np.append(disconnected_line,target_line)
    elif temp.find('OFGT Relay.ElmRelay') >= 0:
        target_gen = int(re.findall(p2,temp)[0])
        print('Target Generator:',target_gen)
        disconnected_gen = np.append(disconnected_gen,target_gen)
    elif temp.find('OVGT Relay.ElmRelay') >= 0:
        target_gen = int(re.findall(p3,temp)[0])
        print('Target Generator:',target_gen)
        disconnected_gen = np.append(disconnected_gen,target_gen)
    elif temp.find('UFLS Relay.ElmRelay') >= 0:
        load = int(re.findall(p4,temp)[0])
        print('Load:',load)
        load_shedding_matrix[load-1] += load_power[load-1]*shedding_percent
    elif temp.find('.ElmSym') >= 0:
        gen_out_of_step = int(re.findall(p5,temp)[0])
    elif temp.find('Generator out of step (pole slip)') >= 0:
        disconnected_gen = np.append(disconnected_gen,gen_out_of_step)

Read line:  - Control conditions for all controllers of interest are fulfilled.
Read line:  info - Element '   SYM1' is local reference in separated area of '   B1'
Read line:  info - Element '   SYM1' is reference in 60.0 Hz-system
Read line:  info - (t=-100:000 ms) Initial conditions calculated.
Read line:  evt  - (t=01:000 s) Grid\Line3.ElmLne:
Read line:  Short-circuit with Fault Impedance Rf = 0.000001 Ohm Xf = 0.000000 Ohm
Read line:  Fault Location: 50.0000 % from 'Grid\B4.ElmTerm'
Read line:  evt  - (t=01:101 s) Grid\B4\Cub_1\Line3 Overcurrent Relay.ElmRelay:
Target line: 3
Read line:  Relay is tripping. 'Open' signal is sent to the connected breaker(s).
Read line:  evt  - (t=01:101 s) Grid\B4\Cub_1\Switch.StaSwitch:
Read line:  Circuit-Breaker Action: 'Open' - 'All phases'.
Read line:  info - (t=01:101 s) Element '   SYM1' is local reference in separated area of '   B1'
Read line:  info - (t=01:101 s) Element '   SYM1' is reference in 60.0 Hz-system
Read line:  evt  - (t=01:73

C:\Users\68075\Anaconda3\envs\pytorch\lib\site-packages\ipykernel_launcher.py:22: ComplexWarning: Casting complex values to real discards the imaginary part


In [52]:
disconnected_line

array([3.])

In [54]:
disconnected_gen

array([2., 1., 2., 1., 2.])

In [55]:
load_shedding_matrix

array([225.,  40.])

In [2]:
pip install xlrd

  Using cached https://files.pythonhosted.org/packages/a6/0c/c2a72d51fe56e08a08acc85d13013558a2d793028ae7385448a6ccdfae64/xlrd-2.0.1-py2.py3-none-any.whl
Note: you may need to restart the kernel to use updated packages.


In [3]:
import csv
import re
import Excel
import Matching
import xlrd
import numpy as np
# import Matlab

In [2]:
p1 = re.compile(r'[=](.*?)[)]', re.S) # (t = 1 s)
p2 = re.compile(r"Grid split into (.*)", re.S) # Grid split into 3 isolated areas
p3 = re.compile(r"Grid\\(.*).ElmLne", re.S) # Grid\Line3.ElmLne
p4 = re.compile(r"Grid\\Line(.*)[-](.*).ElmLne", re.S)
p5 = re.compile(r"Grid\\B(.*)\\Cub_", re.S) # Grid\B2\Cub_4
p6 = re.compile(r"Element (.*) is local reference", re.S) # Element 'SYM1' is local reference'
p7 = re.compile(r"  1 (.*)", re.S)
p8 = re.compile(r"Grid\\(.*).ElmSym", re.S) # 'Grid\SYM2.ElmSym
p9 = re.compile(r"Step: (.*)[)]", re.S)
p10 = re.compile(r"\\Cub_(.*)\\Switch", re.S) # Grid\B2\Cub_4\Switch.StaSwitch
p11 = re.compile(r"G (.*)", re.S)

In [8]:
# Global variables
count = 0
flag = 0
flag2 = 3
Cub1 = ""
Comp1 = ""
Name_of_Local_Reference2 = ""
Switch_Event1 = ""
Unsupplied_Areas1 = ""
State_of_Logic1 = ""
Targeted_Load2 = ""
Percent_of_Shedding1 = ""
Amount_of_Load_Shedding = 0.0
Amount_of_Load_Shedding1 = 0.0
Amount_of_Load_Shedding2 = 0.0
faulty_line = []
faulty_node_overfreq = []
faulty_node_underfreq = []
tripped_line = []
tripped_node_overfreq = []
tripped_node_underfreq = []
tripped_load = []
tripped_LS_balance = []
tripped_LS_tripped_gen = []
tripped_LS_tripped_load = []
tripped_LS_unsupplied = []
failed_matrix = []
faulty_matrix = []
OutOfStep = []
State_of_generator = ""
amount_of_shedding= [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
j = 0
i = 0

# Active Power of Loads
# Loads = Excel.read_excel('IEEE39_Parameters.xlsx', 'Load Parameters', 8, 26, 1, 3, 1.0)
# load_power = [450+217.95j,80+38.75j]
target_load = [450+217.95j,80+38.75j]

# create empty matrix of 512*1 - 12*1
data = [[''], [''], [''], [''], [''], [''],[''], [''], [''], [''], [''], ['']]

name = 'table.csv'
input = 'report.txt'
with open(name, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    f = open(input, 'r')
    for line in f.readlines():
        temp = line.strip()
        print('Read line: ',temp)
        if temp.find('.ElmSym') >= 0:
            Comp = re.findall(p8, temp)
            Comp1 = "".join(Comp)
            print('SYM event')
        elif temp.find('Generator out of step (pole slip)') >= 0:
            t = re.findall(p1, temp)
            t1 = "".join(t)
            data[i].extend([t1, Comp1, 'Out of step', '', ''])
            i = i + 1
            Comp1 = ""
            print('out of step')
#         elif temp.find('Outage Event: Element set to out of service') >= 0:
#             Name_of_Local_Reference2 = ""
#             Unsupplied_Areas1 = ""
#             t = re.findall(p1, temp)
#             t1 = "".join(t)
#             Trip_of_generator = ""
#             if Comp1 != "" :
#                 if Comp1.find("(") == -1 :
#                     gen = re.findall(p11, Comp1)
#                     gen1 = "".join(gen)
#                     name = Comp1 
#                     State_of_generator = '>360 tripped'
#                     Trip_of_generator = Comp1 + ' All tripped'
#                     faulty_node_overfreq.append(Matching.MatchingGen_IEEE39(gen1))
#                 else :
#                     name = Comp1 
#                     State_of_generator = '>360 tripped'
#                     Trip_of_generator = ""
#                 data[i].extend([t1, name, State_of_generator, Trip_of_generator, ''])
#                 i = i + 1
#                 Comp1 = ""
        elif temp.find('.ElmLne') >= 0:
            t = re.findall(p1, temp)
            t1 = "".join(t)
            Comp1 = ""
            if temp.find('evt  -') >= 0:
                Line_Outage = re.findall(p3, temp)
                Line_Outage1 = "".join(Line_Outage)
                Switch_Event = re.findall(p4, temp)
                Switch_Event1 = " ".join(Switch_Event[0])
                Switch_Event2 = Switch_Event1.split(' ')
                row = Matching.Base(Switch_Event2[0], Switch_Event2[1])
                faulty_line.append(row)
                data[i].extend([t1, Line_Outage1, row, '', ''])
                i = i + 1

Read line:  - Control conditions for all controllers of interest are fulfilled.
Read line:  info - Element '   SYM1' is local reference in separated area of '   B1'
Read line:  info - Element '   SYM1' is reference in 60.0 Hz-system
Read line:  info - (t=-100:000 ms) Initial conditions calculated.
Read line:  evt  - (t=01:000 s) Grid\Line3.ElmLne:
Read line:  Short-circuit with Fault Impedance Rf = 0.000001 Ohm Xf = 0.000000 Ohm
Read line:  Fault Location: 50.0000 % from 'Grid\B4.ElmTerm'
Read line:  evt  - (t=01:101 s) Grid\B4\Cub_1\Line3 Overcurrent Relay.ElmRelay:
Read line:  Relay is tripping. 'Open' signal is sent to the connected breaker(s).
Read line:  evt  - (t=01:101 s) Grid\B4\Cub_1\Switch.StaSwitch:
Read line:  Circuit-Breaker Action: 'Open' - 'All phases'.
Read line:  info - (t=01:101 s) Element '   SYM1' is local reference in separated area of '   B1'
Read line:  info - (t=01:101 s) Element '   SYM1' is reference in 60.0 Hz-system
Read line:  evt  - (t=01:731 s) ----------

In [9]:
data

[['', '01:731 s', 'SYM2', 'Out of step', '', ''], [''], [''], [''], [''], ['']]